In [12]:
import openrouteservice as ors
import folium

In [13]:
# enter your openrouteservice api key here as string
ors_key ='5b3ce3597851110001cf6248aadfb7d565d74dc6919eabdc108f87c3'
client = ors.Client(key=ors_key)

## DIRECTIONS
profile argument (string): specifies the mode of transport to use when calculating directions, options: driving-car, driving-hgv, foot-walking, foot-hiking, cycling-regular, cycling-road, cycling-mountain, cycling-electric

In [17]:
# map
map_geocode = folium.Map(location=[23.0225,72.5714], zoom_start=13)

# address
address= input("enter the source point")
address2=input("enter the destination point")
pro=input("walk(foot-walking) or driving car(driving-car)")
# geocode
geocode = client.pelias_search(text=address, focus_point=list(reversed(map_geocode.location)))
geocode2 = client.pelias_search(text=address2, focus_point=list(reversed(map_geocode.location)))
# add marker to map (El Paso Zoo)
for result in geocode['features']:
    folium.Marker(location=list(reversed(result['geometry']['coordinates'])),
                  icon=folium.Icon(icon='building', color='green', prefix='fa'),
                  popup=folium.Popup(result['properties']['name'])).add_to(map_geocode)
    latlon1=list(result['geometry']['coordinates'])
    break
        
for result in geocode2['features']:
    folium.Marker(location=list(reversed(result['geometry']['coordinates'])),
                  icon=folium.Icon(icon='building', color='red', prefix='fa'),
                  popup=folium.Popup(result['properties']['name'])).add_to(map_geocode)
    latlon2=list(result['geometry']['coordinates'])
    break

coordinates = [latlon1,latlon2]

route = client.directions(
    coordinates=coordinates,
    profile=pro,
    format='geojson',
    validate=False,
)
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(map_geocode)
# display map
print(round(route['features'][0]['properties']['segments'][0]['distance']/1000,2), 'kilometers')
print(round(route['features'][0]['properties']['segments'][0]['duration']/60,2), 'minutes\n')

print('directions')
for index, i in enumerate(route['features'][0]['properties']['segments'][0]['steps']):
    print(index+1, i, '\n')
map_geocode

enter the source pointdallas texas
enter the destination pointlos angeles
walk(foot-walking) or driving car(driving-car)driving-car
2321.84 kilometers
1326.04 minutes

directions
1 {'distance': 10.3, 'duration': 1.9, 'type': 11, 'instruction': 'Head southeast', 'name': '-', 'way_points': [0, 1]} 

2 {'distance': 180.8, 'duration': 43.4, 'type': 1, 'instruction': 'Turn right onto Rector Street', 'name': 'Rector Street', 'way_points': [1, 5]} 

3 {'distance': 2114.9, 'duration': 161.7, 'type': 0, 'instruction': 'Turn left onto East Kiest Boulevard', 'name': 'East Kiest Boulevard', 'way_points': [5, 47]} 

4 {'distance': 4043.4, 'duration': 285.1, 'type': 1, 'instruction': 'Turn right onto East Illinois Avenue', 'name': 'East Illinois Avenue', 'way_points': [47, 115]} 

5 {'distance': 1021.1, 'duration': 49.1, 'type': 0, 'instruction': 'Turn left', 'name': '-', 'way_points': [115, 126]} 

6 {'distance': 327.4, 'duration': 23.6, 'type': 13, 'instruction': 'Keep right', 'name': '-', 'way_po

# POIS - PLACES OF INTEREST


In [19]:
# coordinates
p=input("enter your current location ")
b=int(input("buffer (less than 2000)"))
c=int(input("enter search code"))
geocode = client.pelias_search(text=p, focus_point=list(reversed(map_geocode.location)))
for result in geocode['features']:
    point=list(result['geometry']['coordinates'])
    break
geojson = {"type": "point", "coordinates": point}
coordinates = point

# places of interest
pois = client.places(request='pois',
                     geojson=geojson,
                     # buffer searches (in meters) around specified point
                     buffer=b,
                     # hospital: 206, restaurant: 570
                     filter_category_ids=[c])

# map
map_pois = folium.Map(location=list(reversed(coordinates)), tiles='cartodbpositron', zoom_start=14)

# add center point
folium.Marker(list(reversed(coordinates)), icon=folium.Icon(color='red')).add_to(map_pois)

# add search area circle
folium.Circle(radius=2000, location=list(reversed(coordinates)), color='green').add_to(map_pois)

# add markers to map
for poi in pois['features']:
    try:
        p=folium.Popup(poi['properties']['osm_tags']['name'])
    except KeyError:
        p="no data"
    folium.Marker(location=list(reversed(poi['geometry']['coordinates'])),
                  icon=folium.Icon(color='blue'),
                  popup=p).add_to(map_pois)


# display map
map_pois

enter your current location LD college
buffer (less than 2000)2000
enter search code570


# ISOCHRONES
An isochrone map depicts the area accessible from a point within a certain time threshold.

Reachable or accessible areas can be shown graphically by isochrones. Isochrones are imaginary lines passing through all the places that can be reached from a particular origin within a specified travel time.

In [22]:
# coordinates
p=input("enter your current location ")
wht=input("area or population(total_pop)")
r=int(input("your range in seconds"))
geocode = client.pelias_search(text=p, focus_point=list(reversed(map_geocode.location)))
for result in geocode['features']:
    point=list(result['geometry']['coordinates'])
    break

coordinates = [point]

# isochrone
isochrone = client.isochrones(locations=coordinates,
                              range_type='distance',
                              # 900 seconds, 15 minutes
                              range=[r],
                              attributes=[wht])

# map
map_isochrone = folium.Map(location=list(reversed(point)), zoom_start=12)

# add geojson to map with population
population = isochrone['features'][0]['properties'][wht]
folium.GeoJson(isochrone, name='isochrone', tooltip=f' {population:,.0f}').add_to(map_isochrone)

# add marker to map
minutes = isochrone['features'][0]['properties']['value']/60
popup_message = f'outline shows areas reachable within {minutes} minutes'
folium.Marker(list(reversed(point)), popup=popup_message, tooltip='click').add_to(map_isochrone)

# add layer control to map (allows layer to be turned on or off)


# display map
map_isochrone


enter your current location Mumbai
area or population(total_pop)total_pop
your range in seconds4000


In [23]:
p=input("enter your current location ")

geocode = client.pelias_search(text=p, focus_point=list(reversed(map_geocode.location)))
for result in geocode['features']:
    point=list(result['geometry']['coordinates'])
    break
point

enter your current location ahmedabad railway station


[72.601543, 23.0259]

## Matrix
optimized route between multiple points

In [24]:
m = folium.Map(location=[23.008779,72.59977], zoom_start=13)

# Some coordinates in Berlin
coordinates = [[72.509348, 23.034638],[72.50196, 22.98297],[72.59977, 23.008779],[72.509348, 23.034638]]

# The popup will show the ID in the coordinate list. In a non-optimized waypoint order, the waypoints
# would have been visited from ID 0 to ID 3 sequentially.
for idx, coords in enumerate(coordinates):
    folium.Marker(location=list(reversed(coords)),
                 popup=folium.Popup("ID: {}".format(idx))).add_to(m)

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    validate=False,
    optimize_waypoints=True
)

folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

In [11]:
pip install voila

                                              0.0/3.3 MB ? eta -:--:--
                                              0.0/3.3 MB 660.6 kB/s eta 0:00:05
                                              0.0/3.3 MB 660.6 kB/s eta 0:00:05
                                              0.1/3.3 MB 365.7 kB/s eta 0:00:09
     -                                        0.1/3.3 MB 479.1 kB/s eta 0:00:07
     -                                        0.1/3.3 MB 481.4 kB/s eta 0:00:07
     -                                        0.2/3.3 MB 510.2 kB/s eta 0:00:07
     --                                       0.2/3.3 MB 590.8 kB/s eta 0:00:06
     --                                       0.2/3.3 MB 518.8 kB/s eta 0:00:07
     ---                                      0.3/3.3 MB 583.8 kB/s eta 0:00:06
     ---                                      0.3/3.3 MB 610.3 kB/s eta 0:00:05
     ----                                     0.3/3.3 MB 655.4 kB/s eta 0:00:05
     ----                                     0.4